In [1]:
import tensorflow as tf
import numpy as np
import keras
import os

from keras import layers
from keras.optimizers import Adam
from keras.utils import split_dataset

2024-09-20 15:20:44.157201: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
img_width = 124
img_height = 62

In [3]:
path = 'Stacked'

i = 0
slices = []
for fp in sorted(os.listdir(path)):
    if fp.startswith('.'):
        continue
    for f in sorted(os.listdir(os.path.join(path, fp))):
        if f.startswith('.'):
            continue
        with np.load(os.path.join(path, fp, f)) as data:
            features = data['arr_0']
            label = (i, )
            sl = tf.data.Dataset.from_tensor_slices(
                (tf.expand_dims(tf.convert_to_tensor(features, dtype=tf.float32), axis=0),
                 tf.convert_to_tensor((label, ), dtype=tf.int32)))
            slices.append(sl)
    i += 1

shape = len(slices)
print(shape)

1535


In [4]:
ds = tf.data.Dataset.from_tensor_slices(slices)
ds = ds.interleave(
    lambda y: y, cycle_length=1, num_parallel_calls=tf.data.AUTOTUNE
)

del slices

In [5]:
ds_train, ds_test = split_dataset(ds, left_size=0.8, shuffle=True, seed=42)

2024-09-20 15:21:10.078903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [1535]
	 [[{{node Placeholder/_0}}]]
2024-09-20 15:21:10.079076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [1535]
	 [[{{node Placeholder/_0}}]]


In [6]:
batch_size = 2

ds_train = ds_train.batch(batch_size).shuffle(buffer_size=1250)

In [7]:
inputs = layers.Input(shape=(img_height, img_width, 61, 1))

x = layers.Conv3D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.GlobalAveragePooling3D()(x)
x = layers.Dense(units=128, activation='relu')(x)

outputs = layers.Dense(units=4, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 62, 124, 61, 1)]  0         
                                                                 
 conv3d (Conv3D)             (None, 60, 122, 59, 32)   896       
                                                                 
 max_pooling3d (MaxPooling3D  (None, 30, 61, 29, 32)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 30, 61, 29, 32)   128       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 28, 59, 27, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 14, 29, 13, 64)   0     

In [8]:
encodings = {
    'Mild Dementia': 0,
    'Moderate Dementia': 1,
    'Non Demented': 2,
    'Very Mild Dementia': 3
}

def class_weights() -> dict:
    totals = {}
    total = 0

    for fp in sorted(os.listdir('Stacked')):
        if fp.startswith('.'):
            continue
        t = len(os.listdir(os.path.join(path, fp)))
        totals[encodings[str(fp)]] = t
        total += t

    weights = {}
    for _ in totals.keys():
        weights[_] = total/totals[_]

    return weights

class_weight = class_weights()
print(class_weight)

print(shape)

class_weight_3d = np.zeros((shape, 4))
class_weight_3d[:, 0] += class_weight[0]
class_weight_3d[:, 1] += class_weight[1]
class_weight_3d[:, 2] += class_weight[2]
class_weight_3d[:, 3] += class_weight[3]

{0: 9.359756097560975, 1: 34.88636363636363, 2: 1.3929219600725953, 3: 6.822222222222222}
1535


In [9]:
model.compile(optimizer=Adam(learning_rate=.01), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
history = model.fit(ds_train, epochs=10)
model.save('models/hai_3')

2024-09-20 15:21:13.584221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [1228,1]
	 [[{{node Placeholder/_1}}]]
2024-09-20 15:21:13.584676: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [1228,1]
	 [[{{node Placeholder/_1}}]]


Epoch 1/10
 23/614 [>.............................] - ETA: 17:19 - loss: 1.0714 - accuracy: 0.6087

KeyboardInterrupt: 